# First simple tests

## Minimum Example ECDH (without network protocol)

### Keygen Alice

In [2]:
import nacl.public
import time

#start timer Alice
start = time.time()

#Alice generates private key
alice_private = nacl.public.PrivateKey.generate()

#Alice generates public key
alice_public = alice_private.public_key

end = time.time()

time_key_gen_alice = end - start


print("-"*100)
print("Alice generates private key: " + alice_private.encode().hex())
print("-"*100)
print("Alice generates public key: " + alice_public.encode().hex())
print("-"*100)
print(f"Generation key Alice: {time_key_gen_alice:.6f} seconds")
print("-"*100)

----------------------------------------------------------------------------------------------------
Alice generates private key: 190c27f1ad5b7c2e687c753ac75d144035e401a9e7d06188a38c58113dee224d
----------------------------------------------------------------------------------------------------
Alice generates public key: c82ea888d101dba4d66aaa59c34ff9ddc73fdcf188735d3612a15e1199ff6b53
----------------------------------------------------------------------------------------------------
Generation key Alice: 0.000303 seconds
----------------------------------------------------------------------------------------------------


### Keygen Bob

In [28]:
start = time.time()

bob_private = nacl.public.PrivateKey.generate()
bob_public = bob_private.public_key

end = time.time()

time_key_gen_bob = end - start

print("-"*100)
print("Bob generates private key: " + bob_private.encode().hex())
print("-"*100)
print("Bob generates public key: " + bob_public.encode().hex())
print("-"*100)
print(f"Generation key Bob: {time_key_gen_bob:.6f} seconds")
print("-"*100)

----------------------------------------------------------------------------------------------------
Bob generates private key: 3f447c746ef4a69a6b627791901bb58fc61677ee760e8bae66cc35ad8dccf68e
----------------------------------------------------------------------------------------------------
Bob generates public key: f9b4830ab00885c5c4f1129a064c3aaef53578630dbf49e01b07594e9ce9493e
----------------------------------------------------------------------------------------------------
Generation key Bob: 0.000783 seconds
----------------------------------------------------------------------------------------------------


### Calc shared secret

In [32]:
start = time.time()

# Calc shared secrets
alice_box = nacl.public.Box(alice_private, bob_public)
bob_box = nacl.public.Box(bob_private, alice_public)

shared_secret_alice = alice_box.shared_key()
shared_secret_bob = bob_box.shared_key()

end = time.time()
time_shared_secret = end - start

print("-" * 100)
print(f"Key exchange: {time_shared_secret:.6f} seconds")
print("-" * 100)

# Verify that both shared secrets are identical
assert shared_secret_alice == shared_secret_bob, "Key exchange failed: Secrets do not match!"
print(f"Shared Secret: {shared_secret_alice.hex()}")
print("-" * 100)

----------------------------------------------------------------------------------------------------
Key exchange: 0.001988 seconds
----------------------------------------------------------------------------------------------------
Shared Secret: 9799e9e11d376d45577ed578b28c715635bb097a2e97841b481b2452d9ddd041
----------------------------------------------------------------------------------------------------


## Compute Total Time

In [31]:
time_total_ecdh = time_key_gen_alice + time_key_gen_bob + time_shared_secret
print("-" * 100)
print("Total time: " + str(time_total_ecdh) + " seconds")
print("-" *100)

----------------------------------------------------------------------------------------------------
Total time: 0.0029571056365966797 seconds
----------------------------------------------------------------------------------------------------


## Kyber Minimum Example

In [8]:
import oqs
import time

### Generate Key Pair

In [9]:
kem = oqs.KeyEncapsulation('Kyber768')

start = time.time()
public_key = kem.generate_keypair()
end = time.time()

time_kyber_keypair = end - start

print("-" * 100)
print("Time Key Pair: " + str(time_kyber_keypair) + " seconds")
print("-" * 100)

----------------------------------------------------------------------------------------------------
Time Key Pair: 0.015543937683105469 seconds
----------------------------------------------------------------------------------------------------


### Encrypt

In [12]:
start = time.time()
ciphertext, shared_secret_enc = kem.encap_secret(public_key)
end = time.time()

time_kyber_enc = end - start

print("-" * 100)
print(f"Kyber Encryption: {time_kyber_enc:.6f} seconds")
print("-" * 100)

----------------------------------------------------------------------------------------------------
Kyber Encryption: 0.000374 seconds
----------------------------------------------------------------------------------------------------


### Decrypt

In [14]:
start = time.time()
shared_secret_dec = kem.decap_secret(ciphertext)
end = time.time()

time_kyber_decrypt = end - start

print("-" * 100)
print(f"Kyber Decryption: {time_kyber_decrypt:.6f} seconds")
print("-" * 100)

----------------------------------------------------------------------------------------------------
Kyber Decryption: 0.000321 seconds
----------------------------------------------------------------------------------------------------


In [18]:
assert shared_secret_enc == shared_secret_dec, "Key exchange failed: Secrets do not match!"

print("-" * 100)
print(f"Shared Secret: {shared_secret_enc.hex()}")
print("-" * 100)

----------------------------------------------------------------------------------------------------
Shared Secret: 056f85bc3c966990c161ef265029cefdc2e63513f1dcad7fab2e5ec825a1d848
----------------------------------------------------------------------------------------------------


## Simulated Hybrid Handshake

In [5]:
import nacl.public
import oqs
import time

In [4]:
# generate ECDH Key
server_ecdh_private = nacl.public.PrivateKey.generate()
server_ecdh_public = server_ecdh_private.public_key

client_ecdh_private = nacl.public.PrivateKey.generate()
client_ecdh_public = client_ecdh_private.public_key

# generate Kyber Key
server_kyber = oqs.KeyEncapsulation("Kyber768")
server_kyber_public = server_kyber.generate_keypair()

client_kyber = oqs.KeyEncapsulation("Kyber768")
client_kyber_ciphertext, client_kyber_shared_secret = client_kyber.encap_secret(server_kyber_public)

print("Keypairs generated")


# ECDH shared secret
server_box = nacl.public.Box(server_ecdh_private, client_ecdh_public)
shared_secret_ecdh = server_box.shared_key()

 
# Kyber shared secret (decapsulation)
shared_secret_kyber = server_kyber.decap_secret(client_kyber_ciphertext)

# Combine both shared secrets into a hybrid key
hybrid_shared_secret = shared_secret_ecdh + shared_secret_kyber

print("Hybrid shared secret established!")
print(f"Hybrid Key (Hex): {hybrid_shared_secret.hex()}")


# Start timing
start_time = time.time()

# ECDH key exchange
server_box = nacl.public.Box(server_ecdh_private, client_ecdh_public)
shared_secret_ecdh = server_box.shared_key()

# Kyber key exchange (decapsulation)
shared_secret_kyber = server_kyber.decap_secret(client_kyber_ciphertext)

# Combine both shared secrets
hybrid_shared_secret = shared_secret_ecdh + shared_secret_kyber

# End timing
end_time = time.time()
handshake_time = end_time - start_time

print("-" * 50)
print(f"Hybrid Shared Secret: {hybrid_shared_secret.hex()}")
print(f"Handshake Time: {handshake_time:.6f} seconds")
print("-" * 50)

Keypairs generated
Hybrid shared secret established!
Hybrid Key (Hex): 8fcb0e49444e40f7248b4b180967b8b18acc9f1e7656ff8591e9d9b5c96af22b73826217677d5bca46eee206ab6728b7341cab7c602f99572409a81e48907520
--------------------------------------------------
Hybrid Shared Secret: 8fcb0e49444e40f7248b4b180967b8b18acc9f1e7656ff8591e9d9b5c96af22b73826217677d5bca46eee206ab6728b7341cab7c602f99572409a81e48907520
Handshake Time: 0.000710 seconds
--------------------------------------------------
